In [ ]:
# • 神经网络中的激活函数使用平滑变化的sigmoid 函数或ReLU函数。
# • 通过巧妙地使用NumPy多维数组，可以高效地实现神经网络。
# • 机器学习的问题大体上可以分为回归问题和分类问题。
# • 关于输出层的激活函数，回归问题中一般用恒等函数，分类问题中一般用softmax 函数。
# • 分类问题中，输出层的神经元的数量设置为要分类的类别数。
# • 输入数据的集合称为批。通过以批为单位进行推理处理，能够实现高速的运算。

In [ ]:
#![title](img/picture.png)
#P067
# 输出层的神经元数量需要根据待解决的问题来决定。对于分类问题，输出层的神经元数量一般设定为类别的数量。比如，对于某个输入图像，预测是图中的
# 数字0 到9 中的哪一个的问题（10 类别分类问题），可以将输出层的神经元设定为10 个。
#进行手写数字图像的分类。先实现神经网络的“推理处理”。这个推理处理也称为神经网络的前向传播（forward propagation）。


#MNIST Sample MNIST的图像数据是28 像素× 28 像素的灰度图像（1 通道）
# MNIST的图像数据是28 像素× 28 像素的灰度图像（1 通道），各个像素的取值在0 到255 之间。每个图像数据都相应地标有“7”“2”“1”等标签。
# 使用mnist.py中的load_mnist()函数，就可以按下述方式轻松读入MNIST数据
# import sys, os
# os.chdir('/Users/shuhuimeng/ch03/')
# sys.path.append(os.pardir)  
 
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    #由img.shape为（height,width,3）转换为img.size为(width，height)
    pil_img = Image.fromarray(np.uint8(img))#如果需要保存的图片变量是数组array形式的，需要先转换为Image格式，然后再保存
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
# 输出各个数据的形状
print(label)
print(img.shape)# 把图像的形状变成原来的尺寸
img = img.reshape(28, 28)
print(img.shape)
print(x_train.shape) # (60000, 784)

print(t_train.shape) # (60000,)
print(x_test.shape) # (10000, 784)
print(t_test.shape) # (10000,)
img_show(img)

# load_mnist函数以“( 训练图像, 训练标签)，( 测试图像，测试标签)”的形式返回读入的MNIST数据。此外，还可以像load_mnist(normalize=True,

# flatten=True, one_hot_label=False) 这样，设置3 个参数。第1 个参数normalize设置是否将输入图像正规化为0.0～1.0 的值。如果将该参数设置

# 为False，则输入图像的像素会保持原来的0～255。第2 个参数flatten设置是否展开输入图像（变成一维数组）。如果将该参数设置为False，则输入图

# 像为1 × 28 × 28 的三维数组；若设置为True，则输入图像会保存为由784 个元素构成的一维数组。第3 个参数one_hot_label设置是否将标签保存为

# onehot表示（one-hot representation）。one-hot 表示是仅正确解标签为1，其余皆为0 的数组，就像[0,0,1,0,0,0,0,0,0,0]这样。

# 当one_hot_label为False时，只是像7、2这样简单保存正确解标签；当one_hot_label为True时，标签则保存为one-hot 表示。

# 这里需要注意的是，flatten=True时读入的图像是以一列（一维）NumPy数组的形式保存的。因此，显示图像时，需要把它变为原来的28 像素× 28像素的

# 形状。可以通过reshape()方法的参数指定期望的形状，更改NumPy数组的形状。此外，还需要把保存为NumPy数组的图像数据转换为PIL用的数据对象，这

# 个转换处理由Image.fromarray()来完成。

![title](img/18.png)

In [ ]:
#p069
# 神经网络的推理处理
# 我们对这个MNIST数据集实现神经网络的推理处理。神经网络的输入层有784 个神经元，输出层有10 个神经元。输入层的784 这个数字来源于图像大小的
# 28 × 28 = 784，输出层的10 这个数字来源于10 类别分类（数字0 到9，共10 类别）。此外，这个神经网络有2 个隐藏层，第1 个隐藏层有50 个神经元
# ，第2 个隐藏层有100 个神经元。这个50 和100 可以设置为任何值。下面我们先定义get_data()、init_network()、predict()这3 个函数

# load_mnist函数的参数normalize设置成了True。将normalize设置成True后，函数内部会进行转换，将图像的各个像素值除以255，使得数据的值在
# 0.0～1.0 的范围内。像这样把数据限定到某个范围内的处理称为正规化（normalization）。此外，对神经网络的输入数据进行某种既定的转换称为预处理
# （pre-processing）。这里，作为对输入图像的一种预处理，我们进行了正规化。
from dataset.mnist import load_mnist
import pickle
import numpy as np
from activation_function import sigmoid, softmax

def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test
#init_network()会读入保存在pickle 文件sample_weight.pkl中的学习到的权重参数
def init_network():
    with open('sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)

    return network
# predict()函数进行分类。predict()函数以NumPy数组的形式输出各个标签对应的概率。比如输出[0.1, 0.3, 0.2, ..., 0.04]的数组，该数组表示
# “0”的概率为0.1，“1”的概率为0.3，等等。
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y

x, t = get_data()
network = init_network()

accuracy_cnt = 0

for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)#然后，我们取出这个概率列表中的最大值的索引（第几个元素的概率最高），作为预测结果。可以用np.argmax(x)函数取出数组
                    #中的最大值的索引，np.argmax(x)将获取被赋给参数x的数组中的最大值元素的索引。最
    if p == t[i]:
        accuracy_cnt += 1

print('Accuracy:' + str(float(accuracy_cnt)/len(x)))

# y 
# Accuracy:0.9352

![title](img/19.png)

In [ ]:
# 接下來主题是神经网络的学习。这里所说的“学习”是指从训练数据中自动获取最优权重参数的过程，为了使神经网络能进行学习，将导入损失函数这一指标。
# 而学习的目的就是以该损失函数为基准，找出能使它的值达到最小的权重参数。为了找出尽可能小的损失函数的值，本章我们将介绍利用了函数斜率的梯度法。

# 所谓“从数据中学习”，是指可以由数据自动决定权重参数的值。

# 根据“感知机收敛定理”，通过有限次数的学习，线性可分问题是可解的。但是，非线性可分问题则无法通过（自动）学习来解决。
# 从零开始想出一个可以识别數字的算法，不如考虑通过有效利用数据来解决这个问题。一种方案是，先从图像中提取特征量再用机器学习技术学习这些特征量的模式

# 深度学习有时也称为端到端机器学习（end-to-end machinelearning）。这里所说的端到端是指从一端到另一端的意思，也就是从原始数据（输入）中
# 获得目标结果（输出）的意思。

![title](img/20.jpg)
![title](img/21.jpg)

![title](img/22.png)

In [2]:
# 损失函数
# 损失函数是表示神经网络性能的“恶劣程度”的指标，即当前的神经网络对监督数据在多大程度上不拟合，在多大程度上不一致。以“性能的恶劣程度”为指标可能
# 会使人感到不太自然，但是如果给损失函数乘上一个负值，就可以解释为“在多大程度上不坏”，即“性能有多好”。并且，“使性能的恶劣程度达到最小”和“使性
# 能的优良程度达到最大”是等价的，不管是用“恶劣程度”还是“优良程度”，做的事情本质上都是一样的。

In [3]:
# 均方误差可以用作损失函数的函数有很多，其中最有名的是均方误差
# 比如兩組樣本:
# 第一組有以下三個樣本:3，4，5
# 第二組有一下三個樣本:2，4，6
# 這兩組的平均值都是4，但是第一組的三個數值相對更靠近平均值，也就是離散程度
# 小，均方差就是表示這個的。
# 同樣，方差、標準差(方差開根,因為單位不統一)都是表示資料的離散程度的。
# 因為誤差有正有負，如果用算術平均數計算，正負值抵消了好一大半，再除以誤差的個數，則算術
# 平均數會很小。
# 所以將每個點的誤差值取平方值，再計算算術平均數，再開根號。則所有的值為正值，較能顯現出
# 實際狀況。即均方根誤差法。
# 例如：以理想真圓為基準，測量某一圓10 個位置的半徑誤差為：
# +0.12，+0.05，-0.08，+0.03，-0.02，-0.11，-0.06，+0.01，-0.04，+0.05
# 若用算術平均數計算
# =(0.12+0.05-0.08+0.03-0.02-0.11-0.06+0.01+0.04+0.05)/10
# =0.003→幾乎接近真圓，與數據顯現的情況不符
# 若用均方根誤差計算
# =√{[0.122+0.052+(-0.08)2+0.032+(-0.02)2+(-0.11)2+(-0.06)2+0.012+(-0.04)2+0.052]/10}
# =0.0667→較接近數據顯現的情況
# 所取的測量點越多，越接近真實狀況。

In [5]:
import numpy as np
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)
# 设“2”为正确解
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# 例1：“2”的概率最高的情况（0.6）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
B = mean_squared_error(np.array(y), np.array(t))
print(B)

# 例2：“7”的概率最高的情况（0.6）
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
A = mean_squared_error(np.array(y), np.array(t))
print(A)

0.09750000000000003
0.5975


In [6]:
# 交叉熵误差
# 除了均方误差之外，交叉熵误差（cross entropy error）也经常被用作损失函数。
# 假设正确解标签的索引是“2”，与之对应的神经网络的输出是0.6，则交叉熵误差是−log 0.6 = 0.51；若“2”对应的输出是0.1，则交叉熵误差
# 为−log 0.1 = 2.30。也就是说，交叉熵误差的值是由正确解标签所对应的输出结果决定的

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))
    #加上了一个微小值delta。这是因为，当出现np.log(0)时，np.log(0)会变为负无限大的-inf，这样一来就会导致后续计算无法进行。
    #作为保护性对策，添加一个微小值可以防止负无限大的发生
    
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
A = cross_entropy_error(np.array(y), np.array(t))
print(A)

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
B = cross_entropy_error(np.array(y), np.array(t))
print(B)
#2.3025840929945458

0.510825457099338
2.302584092994546


In [ ]:
# MNIST数据集的训练数据有60000 个，如果以全部数据为对象求损失函数的和，则计算过程需要花费较长的时间。再者，如果遇到大数据，数据量会有几百万
# 、几千万之多，这种情况下以全部数据为对象计算损失函数是不现实的。因此，我们从全部数据中选出一部分，作为全部数据的“近似”。神经网络的学习也是
# 从训练数据中选出一批数据（称为mini-batch, 小批量），然后对每个mini-batch 进行学习。比如，从60000 个训练数据中随机选择100 笔，再用这100
# 笔数据进行学习。这种学习方式称为mini-batch 学习。
# min-batchta.py
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = \
load_mnist(normalize=True, one_hot_label=True)
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000, 10)
#那么，如何从这个训练数据中随机抽取10 笔数据呢？
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
#np.random.choice(60000, 10)
print(x_batch)
# 讨论了损失函数，可能有人要问：“为什么要导入损失函数呢？”在神经网络的学习中，寻找最优参数（权重和偏置）时，要寻找使损失函数的值尽可能小的参数。
# 为了找到使损失函数的值尽可能小的地方，需要计算参数的导数（确切地讲是梯度），然后以这个导数为指引，逐步更新参数的值


In [ ]:
#学习算法的实现
# 前提神经网络存在合适的权重和偏置，调整权重和偏置以便拟合训练数据的过程称为“学习”。神经网络的学习分成下面4 个步骤。

# 步骤1（mini-batch）
# 从训练数据中随机选出一部分数据，这部分数据称为mini-batch。我们
# 的目标是减小mini-batch 的损失函数的值。

# 步骤2（计算梯度）
# 为了减小mini-batch 的损失函数的值，需要求出各个权重参数的梯度。
# 梯度表示损失函数的值减小最多的方向。

# 步骤3（更新参数）
# 将权重参数沿梯度方向进行微小更新。

# 步骤4（重复）
# 重复步骤1、步骤2、步骤3。

#p105 P101
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
import matplotlib.pyplot as plt


(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
# 获取mini-batch
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 计算梯度
    #grad = network.numerical_gradient(x_batch, t_batch)高速版
    grad = network.gradient(x_batch, t_batch)
    
    # 更新参数
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    # 记录学习过程
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)


aa = np.array(train_loss_list)
xc = np.arange(len(train_loss_list))

plt.plot(xc, aa)
plt.ylim(0,10)
plt.xlim(0,1000)
plt.show()

# 可以发现随着学习的进行，损失函数的值在不断减小。这是学习正常进行的信号，表示神经网络的权重参数在逐渐拟合数据。也就是说，神经网络的确在学习
# ！通过反复地向它浇灌（输入）数据，神经网络正在逐渐向最优参数靠近。

![title](img/25.png)

In [ ]:
# 训练数据和测试数据的识别精度的推移（横轴的单位是epoch）
# 实线表示训练数据的识别精度，虚线表示测试数据的识别精度。如图所示，随着epoch 的前进（学习的进行），我们发现使用训练数据和测试数据评价的识别精度
# 都提高了，并且，这两个识别精度基本上没有差异（两条线基本重叠在一起）。因此，可以说这次的学习中没有发生过拟合的现象。
#P107
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet
import matplotlib.pyplot as plt


(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)


network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))



markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.xlim(0,30)
plt.legend(loc='lower right')
plt.show()

![title](img/26.png)